# Preprocessing Notebook

In [1]:
import logging
from pathlib import Path
import numpy as np
import pandas as pd

from scdori.pp import config
from scdori.pp.download import (
    download_genome_references
)
from scdori.pp.data_io import (
    create_dir_if_not_exists,
    load_anndata,
    save_processed_datasets
)
from scdori.pp.filtering import (
    intersect_cells,
    remove_mitochondrial_genes
)
from scdori.pp.gene_selection import (
    load_gtf,
    filter_protein_coding_genes,
    compute_hvgs_and_tfs
)
from scdori.pp.peak_selection import keep_promoters_and_select_hv_peaks
from scdori.pp.metacells import create_metacells
from scdori.pp.motif_scanning import (
    run_bedtools_intersect,
    load_motif_database,
    compute_motif_scores
)
from scdori.pp.correlation import compute_in_silico_chipseq
from scdori.pp.utils import create_extended_gene_bed,compute_gene_peak_distance_matrix

logger = logging.getLogger(__name__)

In [ ]:
logging.getLogger().setLevel(config.logging_level)

logger.info("=== Starting multi-ome preprocessing pipeline ===")

## 1. Prepare directories

In [3]:

data_dir = Path(config.data_dir)
genome_dir = Path(config.genome_dir)
motif_dir = Path(config.motif_directory)
out_dir = data_dir / config.output_subdir_name

create_dir_if_not_exists(genome_dir)
create_dir_if_not_exists(motif_dir)
create_dir_if_not_exists(out_dir)

## 2. Download reference genome, gene annotations and chromosome sizes

In [ ]:
download_genome_references(
    genome_dir=genome_dir,
    species=config.species,
    assembly=config.genome_assembly,
    gtf_url=config.gtf_url,
    chrom_sizes_url=config.chrom_sizes_url,
    fasta_url=config.fasta_url
)
# download_motif_database(motif_dir, config.motif_database, config.species)

## 3. Load RNA and ATAC anndata

In [ ]:
data_rna, data_atac = load_anndata(
    data_dir,
    config.rna_adata_file_name,
    config.atac_adata_file_name
)

## 4. Find cells common to both modalities and remove mito genes

In [ ]:
data_rna, data_atac = intersect_cells(data_rna, data_atac)
data_rna = remove_mitochondrial_genes(data_rna, mito_prefix=config.mitochondrial_prefix)


## 5. Optionally filter anndata to protein-coding genes

In [ ]:
# 
gtf_file = genome_dir / "annotation.gtf"
gtf_df = load_gtf(gtf_file)
data_rna = filter_protein_coding_genes(data_rna, gtf_df)

## 6. Selecting highly variable genes and TFs
First we find which TFs have a motif present in the database provided.
User provided sets of genes and TFs are included in final list by default; highly variable computations are performed to obtain remaining genes and TFs

In [ ]:

motif_path = motif_dir / f"{config.motif_database}_{config.species}.meme"
tf_names_all = []
with open(motif_path, "r") as f:
    for line in f:
        if line.startswith("MOTIF"):
            parts = line.strip().split()
            if len(parts) >= 3:
                tf_name = parts[2].split("_")[0].strip("()").strip()
                tf_names_all.append(tf_name)
tf_names_all = sorted(list(set(tf_names_all)))

data_rna, final_genes, final_tfs = compute_hvgs_and_tfs(
    data_rna=data_rna,
    tf_names=tf_names_all,
    user_genes=config.genes_user,
    user_tfs=config.tfs_user,
    num_genes=config.num_genes,
    num_tfs=config.num_tfs,
    min_cells=config.min_cells_per_gene
)

## 7. Create extended gene bed file 
Here we extend the gene body to the user defined genomic window for processing later



In [ ]:

chrom_sizes_path = genome_dir / f"{config.genome_assembly}.chrom.sizes"
extended_genes_bed_df = create_extended_gene_bed(
    gtf_df,
    final_genes + final_tfs,  # if we want to include TF genes too
    window_size=config.window_size,
    chrom_sizes_path=chrom_sizes_path
)

gene_bed_file = out_dir / f"genes_extended_{config.window_size//1000}kb.bed"
extended_genes_bed_df.to_csv(gene_bed_file, sep="\t", header=False, index=False)
logger.info(f"Created extended gene bed => {gene_bed_file}")

In [ ]:
data_atac.var

## 8. Create bed file for all peaks
Of note, that peak anndata var should have chr, start, end and peak_name columns. If not, obtain them

In [11]:

data_atac.var["chr"] = [x.split(":")[0] for x in data_atac.var.index]
data_atac.var["start"] = [int(x.split(":")[1].split("-")[0]) for x in data_atac.var.index]
data_atac.var["end"] = [int(x.split(":")[1].split("-")[1]) for x in data_atac.var.index]
data_atac.var["peak_name"] = data_atac.var.index
all_peaks_bed = out_dir / "peaks_all.bed"
data_atac.var[["chr","start","end","peak_name"]].to_csv(all_peaks_bed, sep="\t", header=False, index=False)

## 9. intersect peaks with extended gene window

Here we subset to peaks which are within a user-defined genomic window of atleast one (selected) gene.

In [ ]:
# 
intersected_bed = out_dir / "peaks_intersected.bed"
run_bedtools_intersect(a_bed=all_peaks_bed, b_bed=gene_bed_file, out_bed=intersected_bed)

peaks_intersected = pd.read_csv(intersected_bed, sep="\t", header=None)
peaks_intersected.columns = ["chr","start","end","peak_name"]
windowed_set = set(peaks_intersected["peak_name"])

# Subset data_atac to these peaks
data_atac = data_atac[:, list(windowed_set)].copy()
logger.info(f"After gene-window filtering => shape={data_atac.shape}")

## 10. Create metacells and store in .obs["leiden"]

Here we obtain metacells using fine-grained leiden clustering on RNA modality. These metacells ar eused to calculate highly variable peaks and to calculate insilico-chipseq scores.

In [ ]:
#
rna_metacell, atac_metacell = create_metacells(
    data_rna, data_atac,
    grouping_key="leiden",
    resolution=config.leiden_resolution,
    batch_key=config.batch_key
)
 # Copy labels
data_atac.obs["leiden"] = data_rna.obs["leiden"]

## 11. Keep promoter peaks and highly variable peaks from the rest => total # = num_peaks

In [ ]:
data_atac = keep_promoters_and_select_hv_peaks(
    data_atac=data_atac,
    total_n_peaks=config.num_peaks,
    cluster_key="leiden",
    promoter_col=config.promoter_col  # column in data_atac.var
)

logger.info(f"Final shape after combining promoters + HV => {data_atac.shape}")

## 12. Save processed ATAC and RNA data

In [ ]:
save_processed_datasets(data_rna, data_atac, out_dir)

## 13. Make bed file for final set of peaks (post selection)

In [16]:
data_atac.var["chr"] = [v.split(":")[0] for v in data_atac.var_names]
data_atac.var["start"] = [int(v.split(":")[1].split("-")[0]) for v in data_atac.var_names]
data_atac.var["end"] = [int(v.split(":")[1].split("-")[1]) for v in data_atac.var_names]
data_atac.var["peak_name"] = data_atac.var_names
peaks_bed = out_dir / "peaks_selected.bed"
data_atac.var[["chr","start","end","peak_name"]].to_csv(
    peaks_bed, sep="\t", header=False, index=False
)

## 14. Compute motif matches for peaks

We use FIMO module from tangermeme (https://tangermeme.readthedocs.io/en/latest/tutorials/Tutorial_D1_FIMO.html) to score the motifs

In [ ]:
motif_path = Path(config.motif_directory) / f"{config.motif_database}_{config.species}.meme"
pwms_sub, key_to_tf = load_motif_database(motif_path, final_tfs)
fasta_path = genome_dir / f"{config.genome_assembly}.fa"
df_motif_scores = compute_motif_scores(
    bed_file=peaks_bed,
    fasta_file=fasta_path,
    pwms_sub=pwms_sub,
    key_to_tf=key_to_tf,
    n_peaks=data_atac.shape[1],
    window=500,threshold= config.motif_match_pvalue_threshold
)
df_motif_scores=df_motif_scores[final_tfs]  

In [18]:
df_motif_scores.to_csv(out_dir / "motif_scores.tsv", sep="\t")

## 15. compute insilico-chipseq
We first subset the previously computed ATAC metacell matrix to selected peaks and use it calculate correlation of TF-peak expression-accesibility. These correlations are thresholded based on an empirically determined cutoff ( from non-motif matched peaks per TF) and then multiplied by motif matching scores from FIMO to obtain insilico-chipseq scores ( adapted from https://www.biorxiv.org/content/10.1101/2022.06.15.496239v1 and DiFFTF https://pubmed.ncbi.nlm.nih.gov/31801079/)

In [ ]:
# 14) Recompute metacells for correlation with selected peaks
    #     Or subset existing atac_metacell to the new set of peaks
# then compute insilico-chipseq
atac_metacell = atac_metacell[:, data_atac.var_names].copy()
tf_mask = rna_metacell.var["gene_type"] == "TF"
rna_matrix = rna_metacell.X[:, tf_mask]  # shape=(n_meta, n_tfs)
atac_matrix = atac_metacell.X  # shape=(n_meta, n_peaks)

insilico_chipseq_act, insilico_chipseq_rep = compute_in_silico_chipseq(
    atac_matrix=atac_matrix,
    rna_matrix=rna_matrix,
    motif_scores=df_motif_scores,
    percentile=config.correlation_percentile,
    n_bg=config.n_bg_peaks_for_corr
)
np.save(out_dir / "insilico_chipseq_act.npy", insilico_chipseq_act)
np.save(out_dir / "insilico_chipseq_rep.npy", insilico_chipseq_rep)

## 16. Compute distance matrix between peaks and genes

In [ ]:
# distance is set to 0 if the peak midpoint is within gene-body or promoter (5kb upstream of TSS by default)
# distance is -1 if peak-gene pairs on different chromosomes

data_atac.var["index_int"] = range(data_atac.shape[1])
selected_peak_indices = data_atac.var["index_int"].values

# Subset GTF to final genes
gene_info = gtf_df[gtf_df.feature == "gene"].drop_duplicates("gene_name")
gene_info['gene']=gene_info['gene_name'].values
gene_info = gene_info.set_index("gene_name")
gene_info = gene_info.loc[
    data_rna.var_names.intersection(gene_info.index)
]

gene_info["chr"] = gene_info["seqname"]  # rename col for consistency
# Create gene_coordinates_intersect with necessary columns
gene_info = gene_info[[
    "chr", "start", "end", "strand","gene"
]].copy()
gene_info.columns = ["chr_gene", "start", "end", "strand","gene"]

dist_matrix = compute_gene_peak_distance_matrix(
    data_rna=data_rna,
    data_atac=data_atac,
    gene_coordinates_intersect=gene_info
    
)
np.save(out_dir / "gene_peak_distance_raw.npy", dist_matrix)

In [ ]:
dist_matrix # -1 denotes peaks on different chromosome

## 17. obtaining distance based decay terms to initialise peak-gene matrix for training scDoRI 

In [23]:
dist_matrix[dist_matrix < 0 ]= 1e8
dist_matrix = np.exp(-1 * dist_matrix.astype(float) / config.peak_distance_scaling_factor)
dist_matrix = np.where(dist_matrix < config.peak_distance_min_cutoff, 0, dist_matrix)
np.save(out_dir / "gene_peak_distance_exp.npy", dist_matrix)

## 18. Final Logging, completed preprocessing

In [ ]:
logger.info("=== Pipeline completed successfully ===")